# American and Bermudan Option pricer

In [1]:
# Packages used
import numpy as np
import matplotlib.pyplot as plt
from scipy import odr
from helper import GBM_Euler, value_option_schwarz, value_option_schwarz_test, bermudian_pricer

## Simulations Run with Data from paper

American option should be priced at .1144

In [42]:
# Parameters
K=1.1; M=4; r=.06; realizations=8

# Matrix
path_matrix = np.array([
[1.00, 1.09, 1.08, 1.34],
[1.00, 1.16, 1.26, 1.54],
[1.00, 1.22, 1.07, 1.03],
[1.00, 0.93, 0.97, 0.92],
[1.00, 1.11, 1.56, 1.52],
[1.00, .76, 0.77, 0.90],
[1.00, 0.92, 0.84, 1.01],
[1.00, 0.88, 1.22, 1.34]])

In [44]:
# Generate cash flows under Longstaff-Schwarz
cash_flows = value_option_schwarz(M,K,path_matrix, r, realizations, option="put")

# Discount cash flows
for time in range(cash_flows.shape[1]):
    cash_flows[:,time]*=np.exp(-r*time)
print(f'Price of American Option is: {np.sum(cash_flows[0:])/realizations}')

Price of American Option is: 0.11665163113684986


In [54]:
cash_flows_test = value_option_schwarz_test(M,K,path_matrix, r, realizations, option="put")

# Discount cash flows
for time in range(cash_flows.shape[1]):
    cash_flows_test[:,time]*=np.exp(-r*time)
print(f'Price of American Option is: {np.sum(cash_flows_test[0:])/realizations}')    

Price of American Option is: 0.11443433004505696


In [56]:
# Unit test Bermudian pricer by giving it same amount of exericse options as American
exercise_dates = [1,2,3]
cash_flow_bermudan = bermudian_pricer(M, K, path_matrix, r, realizations, exercise_dates, option="put")

for time in range(cash_flow_bermudan.shape[1]):
    cash_flow_bermudan[:,time] *= np.exp(-r*time)
print(f'Price of Bermudan Option is: {np.sum(cash_flow_bermudan[0:])/realizations}')    

Price of Bermudan Option is: 0.11606347518346377


In [60]:
# Fewer exercise points should lead to lower option price
exercise_dates = [2,3]
cash_flow_bermudan = bermudian_pricer(M, K, path_matrix, r, realizations, exercise_dates, option="put")

for time in range(cash_flow_bermudan.shape[1]):
    cash_flow_bermudan[:,time] *= np.exp(-r*time)
print(f'Price of Bermudan Option is: {np.sum(cash_flow_bermudan[0:])/realizations}')    

Price of Bermudan Option is: 0.10123968736030678


## Simulation based on GBM

In [68]:
# Variables used
T = 1
K = 100
S = 100
M = 48
sigma = 0.2
r = 0.06/M
realizations = 100
exercise_dates = np.array([6, 12, 18, 24, 30, 36]) - 1

In [69]:
# Generate stock scenarios
s_all = np.array([np.array(GBM_Euler(T, S, sigma, r, M)) for x in range(realizations)])

In [70]:
# Generate path matrix
path_matrix = np.zeros((realizations, M))
for realization in range(realizations):
    path_matrix[realization,:] = s_all[realization]

In [73]:
option_cash_flow = value_option_schwarz(M,K,path_matrix, r, realizations,option="put")

for time in range(option_cash_flow.shape[1]):
    option_cash_flow[:,time]*=np.exp(-r*time)
    
print(np.sum(option_cash_flow[1:])/realizations)

2.8860019125550376


In [74]:
cash_flows_bermudan = bermudian_pricer(M, K, path_matrix, r, realizations, exercise_dates, option = "put")

In [84]:
for time in range(cash_flows_bermudan.shape[1]):
    
    cash_flows_bermudan[:,time]*=np.exp(-r*exercise_dates[time])

print(np.sum(cash_flows_bermudan[1:])/realizations)

3.557788254727098


In [85]:
time


5

In [87]:
exercise_dates[time]

35